In [1]:
import requests
import json 
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup 

In [2]:
import requests
import websocket
import time
import json

In [3]:
import time
from selenium import webdriver

import time
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup as bs

import requests

# html list 만들기

In [4]:
url = 'http://www.foodsafetykorea.go.kr/portal/healthyfoodlife/searchHomeHFDetail.do'
url_list = []
for i in range(2022021000213825,2022021000213835):
    for j in range(1,7):
        payload = {
            'prdlstReportLedgNo': i,
            'search_code': '01',
            'start_idx': j,
            'menu_no': '2823',
            'menu_grp': 'MENU_NEW01'
        }
        
        sess = requests.Session()
        respond1 = sess.post(url,data=payload)
        url_list.append(respond1)
    

In [5]:
len(url_list)

60

# 데이터 파싱

In [10]:
datalist = []
for url in url_list:
    html = url.text
    soup = bs(html, "html.parser")
    
    업소명 = soup.select('div.page-container > article > table > tbody > tr:nth-child(1)> td')
    제품명 = soup.select('div.page-container > article > table > tbody > tr:nth-child(2)> td')
    신고번호 = soup.select('div.page-container > article > table > tbody > tr:nth-child(3)> td')
    등록일자 = soup.select('div.page-container > article > table > tbody > tr:nth-child(4)> td')
    유통기한 = soup.select('div.page-container > article > table > tbody > tr:nth-child(5)> td')
    성상 = soup.select('div.page-container > article > table > tbody > tr:nth-child(6)> td')
    섭취량 = soup.select('div.page-container > article > table > tbody > tr:nth-child(7)> td')
    포장재질 = soup.select('div.page-container > article > table > tbody > tr:nth-child(8)> td')
    보존 = soup.select('div.page-container > article > table > tbody > tr:nth-child(9)> td')
    섭취시주의사항 = soup.select('div.page-container > article > table > tbody > tr:nth-child(10)> td')
    기능성내용 = soup.select('div.page-container > article > table > tbody > tr:nth-child(11)> td')
    기준 = soup.select('div.page-container > article > table > tbody > tr:nth-child(12)> td')

    for item in zip(업소명, 제품명, 신고번호, 등록일자, 유통기한, 성상, 섭취량, 포장재질, 보존, 섭취시주의사항, 기능성내용, 기준):
        datalist.append(
            {'업소명' : item[0].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '제품명' : item[1].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '신고번호' : item[2].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '등록일자' : item[3].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '유통기한' : item[4].text.replace('\n', '').replace('\t', '').replace('  ', ''), 
             '성상' : item[5].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '섭취량' : item[6].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '포장재질' : item[7].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '보존' : item[8].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '섭취시주의사항' : item[9].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '기능성내용' : item[10].text.replace('\n', '').replace('\t', '').replace('  ', ''),
             '기준' : item[11].text.replace('\n', '').replace('\t', '').replace('  ', ''),
            })
    
    
data= pd.DataFrame(datalist)

In [11]:
# 건강기능식품만 남기기, 중복제거
data2 = data.query('성상!=""').reset_index(drop=True)
data3 = data2.drop_duplicates(['제품명'], keep='first') #중복제거
data3

,업소명,제품명,신고번호,등록일자,유통기한,성상,섭취량,포장재질,보존,섭취시주의사항,기능성내용,기준
0,주)팜크로스,혈당엔 바나바,200400151101082,2022-05-30,제조일로부터 24개월,"고유의 향미가 있고 이미·이취가 없는 흰색, 검은색 점박이를 포함한 회황색의 장방형...","1일 1회, 1회 1정을 물과 함께 섭취하십시오.","폴리에틸렌테레프탈레이트(PET), 알루미늄박폴리에틸렌(PE), 고밀도폴리에틸렌(HD...",직사광선 및 고온다습한 곳을 피하여 건조하고 서늘한 곳에 보관하십시오.,(1) 당뇨병이 있는 경우 섭취 전 전문가와 상담할 것(2) 이상사례 발생 시 섭취...,①식후 혈당상승 억제에 도움을 줄 수 있음항산화 작용을 하여 유해산소로부터 세포를 ...,"1. 성상 : 고유의 향미가 있고 이미·이취가 없는 흰색, 검은색 점박이를 포함한 ..."
6,(주)팜투팜2공장,가르시니아 라이트미(美)다이어트,2018000391549,2022-05-30,제조일로부터 18개월,"고유의 향미가 있고 이미, 이취가 없는 밝은 회황색의 분말을 함유한 흰색의 경질캡슐","1일 2회, 1회 2캡슐을 물과 함께 섭취 하십시오.","폴리에틸렌테레프탈레이트(PET), 폴리에틸렌(HDPE), 폴리프로필렌(PP), 염화...",1) 직사광선 및 고온다습한 곳을 피하여 건조하고 서늘한 실온에 보관하십시오.2) ...,① 질환이 있거나 의약품 복용 시 전문가와 상담 하십시오. ② 알레르기 체질 등은 ...,[프로바이오틱스] 유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 ...,"1. 성상: 고유의 향미가 있고 이미, 이취가 없는 밝은 회황색의 분말을 함유한 흰..."
